In [ ]:
%%capture
#export
import torch
import torch.nn as nn
from fastai2.basics import *
from fastcore.test import *

from transformers.modeling_utils import PreTrainedModel, top_k_top_p_filtering, BeamHypotheses

In [ ]:
# default_exp generated_lm

# GeneratedLM
> Language Model or Decoder with Generate function

## FakeDecoder, FakeLM for Test

In [ ]:
class FakeDecoder(nn.Module):
    ''' with memory not support past'''
    def __init__(self, tgt_vocab_size):
        super().__init__()
        self.tgt_vocab_size = tgt_vocab_size
    def forward(self, tgt, momory, **kwargs):
        ''' 
        inputs: (tgt, memory)
            tgt: (b, tgt_seq_len)
            memory: (b, src_seq_len, embed_dim)
        returns: logits, others
            logits: (b, tgt_seq_len, tgt_vocab_size)
        '''
        logits = torch.randn((*tgt.shape, self.tgt_vocab_size))
        return logits, None
    
class FakeLM(nn.Module):
    ''' without memory support past '''
    def __init__(self, tgt_vocab_size):
        super().__init__()
        self.tgt_vocab_size = tgt_vocab_size
    def forward(self, tgt, past=None, **kwargs):
        ''' 
        if past==None:
            inputs: (tgt)
                tgt: (b, tgt_seq_len)
            returns: logits, presents, others
                logits: (b, tgt_seq_len, tgt_vocab_size)
                presents: List of (2, b, ...)
        else:
            inputs: (tgt, past)
                tgt: (b, 1)   
                past: List of (2, bs, num_heads, tgt_seq_len-1, ..)
            returns: logits, presents, others
                logits: (b, tgt_seq_len, tgt_vocab_size)
                presents: List of (2, bs, num_heads, tgt_seq_len, ..)
        '''
        if past is None:
            b = tgt.shape[0]
            tgt_seq_len = tgt.shape[1]
            logits = torch.randn((b, tgt_seq_len, self.tgt_vocab_size))
            presents = [torch.randn((2, b, 12, tgt_seq_len, 16))] * 6
        else:
            b = tgt.shape[0]
            tgt_seq_len = past[0].shape[3]+1
            logits = torch.randn((b, tgt_seq_len, self.tgt_vocab_size))
            presents = [torch.randn((2, b, 12, tgt_seq_len, 16))] * 6
        return logits, presents, None

In [ ]:
bs = 3
tgt_seq_len = 10
tgt_vocab_size = 20
memory = torch.randn((bs, 9, 9))
past = [torch.randn((2, bs, 12, tgt_seq_len-1, 16))] * 6
pad_token_id=0
eos_token_id=tgt_vocab_size-1
bos_token_id=tgt_vocab_size-2

In [ ]:
decoder = FakeDecoder(tgt_vocab_size)
lm = FakeLM(tgt_vocab_size)

tgt = torch.randint(0, tgt_vocab_size-2, (bs, tgt_seq_len))
test_eq(decoder(tgt, memory)[0].shape, (bs, tgt_seq_len, tgt_vocab_size))
test_eq(lm(tgt)[0].shape, (bs, tgt_seq_len, tgt_vocab_size))
test_eq(lm(tgt)[1][0].shape, (2, bs, 12, tgt_seq_len, 16))

tgt = torch.randint(0, tgt_vocab_size-2, (bs, tgt_seq_len))
test_eq(lm(tgt, past)[0].shape, (bs, tgt_seq_len, tgt_vocab_size))
test_eq(lm(tgt, past)[1][0].shape, (2, bs, 12, tgt_seq_len, 16))

## GeneratedLM

In [ ]:
# export
class GeneratedLM():
    def __init__(self, lm, vocab_size, support_past=False):
        '''
        Your lm's forward function should be this format:
        if support_past==False
            inputs: (tgt, *model_otherargs, **model_otherkwargs)
                tgt: (bs, tgt_seq_len)
            returns: logits, others
                logits: (bs, tgt_seq_len, tgt_vocab_size)
        else:
            inputs: (tgt, past, *model_otherargs, **model_otherkwargs)
                tgt: (bs, 1)
                past: The infos for quickly generate sentence.  List of (2, bs, num_heads, tgt_seq_len-1, ..)
            returns: logits, presents, others
                logits: (bs, tgt_seq_len, tgt_vocab_size)
                presents: List of (2, bs, num_heads, tgt_seq_len, ..)
        '''
        self.lm = lm
        self.vocab_size = vocab_size
        self.support_past = support_past

### _generate_no_beam_search

In [ ]:
# export
@patch
def _generate_no_beam_search(
    self:GeneratedLM,
    tgt, # (bs, tgt_seq_len)
    max_length,
    do_sample,
    temperature,
    top_k,
    top_p,
    repetition_penalty,
    pad_token_id,
    eos_token_ids,
    model_otherargs=[],
    model_otherkwargs={},
):
    """ Generate sequences for each example without beam search (num_beams == 1).
        All returned sequence are generated independantly.
    """
    # current position / max lengths / length of generated sentences / unfinished sentences
    batch_size = tgt.shape[0]
    cur_len = tgt.shape[1]
    unfinished_sents = tgt.new(batch_size).fill_(1)

    past = None

    while cur_len < max_length:
#         print('===================================:', cur_len)
#         print(tgt)
        
        if self.support_past==False:
            outputs = self.lm(tgt, *model_otherargs, **model_otherkwargs)
        else:
            model_inputs = tgt if past is None else tgt[:, -1].unsqueeze(-1)
            outputs = self.lm(model_inputs, past, *model_otherargs, **model_otherkwargs) 
        next_token_logits = outputs[0][:, -1, :]
        
        # if model has past, then set the past variable to speed up decoding
        if self.support_past:
            past = outputs[1]

        # repetition penalty from CTRL paper (https://arxiv.org/abs/1909.05858)
        if repetition_penalty != 1.0:
            for i in range(batch_size):
                for previous_token in set(tgt[i].tolist()):
                    # if score < 0 then repetition penalty has to multiplied to reduce the previous token probability
                    if next_token_logits[i, previous_token] < 0:
                        next_token_logits[i, previous_token] *= repetition_penalty
                    else:
                        next_token_logits[i, previous_token] /= repetition_penalty

        if do_sample:
            # Temperature (higher temperature => more likely to sample low probability tokens)
            if temperature != 1.0:
                next_token_logits = next_token_logits / temperature
            # Top-p/top-k filtering
            next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            # Sample
            next_token = torch.multinomial(F.softmax(next_token_logits, dim=-1), num_samples=1).squeeze(1)
        else:
            # Greedy decoding
            next_token = torch.argmax(next_token_logits, dim=-1)

        # update generations and finished sentences
        tokens_to_add = next_token * unfinished_sents + pad_token_id * (1 - unfinished_sents)
        tgt = torch.cat([tgt, tokens_to_add.unsqueeze(-1)], dim=-1)
        for eos_token_id in eos_token_ids:
            unfinished_sents.mul_(tokens_to_add.ne(eos_token_id).long())
        cur_len = cur_len + 1

        # stop when there is a </s> in each sentence, or if we exceed the maximul length
        if unfinished_sents.max() == 0:
            break

    # add eos_token_ids to unfinished sentences
    if cur_len == max_length:
        tgt[:, -1].masked_fill_(unfinished_sents.to(dtype=torch.bool), eos_token_ids[0])

    return tgt # (bs, (tgt_seq_len <= ? <= max_length))

In [ ]:
max_length=20
generate_args = dict(   
    max_length=max_length,
    do_sample=True,
    temperature=0.1,
    top_k=3,
    top_p=0.5,
    repetition_penalty=1.2,
    pad_token_id=pad_token_id,
    eos_token_ids=[eos_token_id],
)

# With memory, Without past
generated_decoder = GeneratedLM(decoder, tgt_vocab_size, support_past=False)
tgt = torch.zeros((bs, 1), dtype=torch.long).fill_(bos_token_id)
result = generated_decoder._generate_no_beam_search(tgt, **generate_args, model_otherargs=[memory])
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= 1

# Without memory, With past
generated_lm = GeneratedLM(lm, tgt_vocab_size, support_past=True)
tgt = torch.randint(0, tgt_vocab_size-2, (bs, tgt_seq_len))
result = generated_lm._generate_no_beam_search(tgt, **generate_args)
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= tgt_seq_len

# Without memory, Without past
generated_lm = GeneratedLM(lm, tgt_vocab_size, support_past=False)
tgt = torch.randint(0, tgt_vocab_size-2, (bs, tgt_seq_len))
result = generated_lm._generate_no_beam_search(tgt, **generate_args)
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= tgt_seq_len

### _generate_beam_search

In [ ]:
# export
@patch
def _generate_beam_search(
    self:GeneratedLM,
    tgt, # (b, tgt_seq_len)
    max_length,
    do_sample,
    temperature,
    top_k,
    top_p,
    repetition_penalty,
    pad_token_id,
    eos_token_ids,
    length_penalty,
    num_beams,
    vocab_size,
    model_otherargs=[],
    model_otherkwargs={},
):
    """ Generate sequences for each example with beam search.
    """
    batch_size = tgt.shape[0]
    cur_len = tgt.shape[1]
    # Expand input to num beams
    tgt = tgt.unsqueeze(1).expand(batch_size, num_beams, cur_len)
    tgt = tgt.contiguous().view(batch_size * num_beams, cur_len)  # (batch_size * num_beams, cur_len)

    # generated hypotheses
    generated_hyps = [
        BeamHypotheses(num_beams, max_length, length_penalty, early_stopping=False) for _ in range(batch_size)
    ]

    # scores for each sentence in the beam
    beam_scores = torch.zeros((batch_size, num_beams), dtype=torch.float, device=tgt.device)
    beam_scores[:, 1:] = -1e9
    beam_scores = beam_scores.view(-1)  # shape (batch_size * num_beams,)

    # cache compute states
    past = None

    # done sentences
    done = [False for _ in range(batch_size)]

    while cur_len < max_length:
        if self.support_past==False:
            outputs = self.lm(tgt, *model_otherargs, **model_otherkwargs)
        else:
            model_inputs = tgt if past is None else tgt[:, -1].unsqueeze(-1)
            outputs = self.lm(model_inputs, past, *model_otherargs, **model_otherkwargs) 
#         outputs = self.lm(tgt, *model_otherargs, **model_otherkwargs) if self.support_past==False else self.lm(tgt, past, *model_otherargs, **model_otherkwargs) # (batch_size * num_beams, cur_len, vocab_size)
        scores = outputs[0][:, -1, :]  # (batch_size * num_beams, vocab_size)

        # if model has past, then set the past variable to speed up decoding
        if self.support_past:
            past = outputs[1]

        # repetition penalty (from CTRL paper https://arxiv.org/abs/1909.05858)
        if repetition_penalty != 1.0:
            for i in range(batch_size * num_beams):
                for previous_token in set(tgt[i].tolist()):
                    # if score < 0 then repetition penalty has to multiplied to reduce the previous token probability
                    if scores[i, previous_token] < 0:
                        scores[i, previous_token] *= repetition_penalty
                    else:
                        scores[i, previous_token] /= repetition_penalty

        if do_sample:
            # Temperature (higher temperature => more likely to sample low probability tokens)
            if temperature != 1.0:
                scores = scores / temperature
            # Top-p/top-k filtering
            scores = top_k_top_p_filtering(
                scores, top_k=top_k, top_p=top_p, min_tokens_to_keep=2
            )  # (batch_size * num_beams, vocab_size)
            # Sample 2 next words for each beam (so we have some spare tokens and match output of greedy beam search)
            next_words = torch.multinomial(F.softmax(scores, dim=-1), num_samples=2)  # (batch_size * num_beams, 2)
            # Compute next scores
            _scores = F.log_softmax(scores, dim=-1)  # (batch_size * num_beams, vocab_size)
            _scores = torch.gather(_scores, -1, next_words)  # (batch_size * num_beams, 2)
            next_scores = _scores + beam_scores[:, None].expand_as(_scores)  # (batch_size * num_beams, 2)
            # Match shape of greedy beam search
            next_words = next_words.view(batch_size, 2 * num_beams)  # (batch_size, 2 * num_beams)
            next_scores = next_scores.view(batch_size, 2 * num_beams)  # (batch_size, 2 * num_beams)
        else:
            # do greedy beam search
            scores = F.log_softmax(scores, dim=-1)  # (batch_size * num_beams, vocab_size)
            assert scores.size() == (batch_size * num_beams, vocab_size)
            # Add the log prob of the new beams to the log prob of the beginning of the sequence (sum of logs == log of the product)
            _scores = scores + beam_scores[:, None].expand_as(scores)  # (batch_size * num_beams, vocab_size)
            # re-organize to group the beam together (we are keeping top hypothesis accross beams)
            _scores = _scores.view(batch_size, num_beams * vocab_size)  # (batch_size, num_beams * vocab_size)
            next_scores, next_words = torch.topk(_scores, 2 * num_beams, dim=1, largest=True, sorted=True)

        assert next_scores.size() == next_words.size() == (batch_size, 2 * num_beams)

        # next batch beam content
        # list of (batch_size * num_beams) tuple(next hypothesis score, next word, current position in the batch)
        next_batch_beam = []

        # for each sentence
        for batch_ex in range(batch_size):

            # if we are done with this sentence
            done[batch_ex] = done[batch_ex] or generated_hyps[batch_ex].is_done(next_scores[batch_ex].max().item())
            if done[batch_ex]:
                next_batch_beam.extend([(0, pad_token_id, 0)] * num_beams)  # pad the batch
                continue

            # next sentence beam content
            next_sent_beam = []

            # next words for this sentence
            for idx, score in zip(next_words[batch_ex], next_scores[batch_ex]):

                # get beam and word IDs
                beam_id = idx // vocab_size
                word_id = idx % vocab_size

                # end of sentence, or next word
                if word_id.item() in eos_token_ids or cur_len + 1 == max_length:
                    generated_hyps[batch_ex].add(
                        tgt[batch_ex * num_beams + beam_id, :cur_len].clone(), score.item()
                    )
                else:
                    next_sent_beam.append((score, word_id, batch_ex * num_beams + beam_id))

                # the beam for next step is full
                if len(next_sent_beam) == num_beams:
                    break

            # update next beam content
            assert len(next_sent_beam) == 0 if cur_len + 1 == max_length else num_beams
            if len(next_sent_beam) == 0:
                next_sent_beam = [(0, pad_token_id, 0)] * num_beams  # pad the batch
            next_batch_beam.extend(next_sent_beam)
            assert len(next_batch_beam) == num_beams * (batch_ex + 1)

        # sanity check / prepare next batch
        assert len(next_batch_beam) == batch_size * num_beams
        beam_scores = beam_scores.new([x[0] for x in next_batch_beam])
        beam_words = tgt.new([x[1] for x in next_batch_beam])
        beam_idx = tgt.new([x[2] for x in next_batch_beam])

        # re-order batch
        tgt = tgt[beam_idx, :]
        tgt = torch.cat([tgt, beam_words.unsqueeze(1)], dim=-1)

        # re-order internal states
        if past is not None:
            reordered_past = []
            for layer_past in past:
                # get the correct batch idx from layer past batch dim
                # batch dim of `past` and `mems` is at 2nd position
                reordered_layer_past = [layer_past[:, i].unsqueeze(1).clone().detach() for i in beam_idx]
                reordered_layer_past = torch.cat(reordered_layer_past, dim=1)
                # check that shape matches
                assert reordered_layer_past.shape == layer_past.shape
                reordered_past.append(reordered_layer_past)
            past = tuple(reordered_past)

        # update current length
        cur_len = cur_len + 1

        # stop when we are done with each sentence
        if all(done):
            break

    # visualize hypotheses
    # print([len(x) for x in generated_hyps], cur_len)
    # globals().update( locals() );
    # !import code; code.interact(local=vars())
    # for ii in range(batch_size):
    #     for ss, ww in sorted(generated_hyps[ii].hyp, key=lambda x: x[0], reverse=True):
    #         print("%.3f " % ss + " ".join(self.dico[x] for x in ww.tolist()))
    #     print("")

    # select the best hypotheses
    tgt_len = tgt.new(batch_size)
    best = []

    for i, hypotheses in enumerate(generated_hyps):
        best_hyp = max(hypotheses.hyp, key=lambda x: x[0])[1]
        tgt_len[i] = len(best_hyp) + 1  # +1 for the <EOS> symbol
        best.append(best_hyp)

    # generate target batch
    decoded = tgt.new(batch_size, tgt_len.max().item()).fill_(pad_token_id)
    for i, hypo in enumerate(best):
        decoded[i, : tgt_len[i] - 1] = hypo
        decoded[i, tgt_len[i] - 1] = eos_token_ids[0]

    return decoded # (b, (tgt_seq_len <= ? <= max_length))

In [ ]:
max_length=20
generate_args = dict(   
    max_length=max_length,
    do_sample=True,
    temperature=0.1,
    top_k=3,
    top_p=0.5,
    repetition_penalty=1.2,
    pad_token_id=pad_token_id,
    eos_token_ids=[eos_token_id],
    length_penalty=1,
    num_beams=4,
    vocab_size=tgt_vocab_size,
)

# With memory, Without past
generated_decoder = GeneratedLM(decoder, tgt_vocab_size, support_past=False)
tgt = torch.zeros((bs, 1), dtype=torch.long).fill_(bos_token_id)
result = generated_decoder._generate_beam_search(tgt, **generate_args, model_otherargs=[memory])
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= 1

# Without memory, With past
generated_lm = GeneratedLM(lm, tgt_vocab_size, support_past=True)
tgt = torch.randint(0, tgt_vocab_size-2, (bs, tgt_seq_len))
result = generated_lm._generate_beam_search(tgt, **generate_args)
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= tgt_seq_len

# Without memory, Without past
generated_lm = GeneratedLM(lm, tgt_vocab_size, support_past=False)
tgt = torch.randint(0, tgt_vocab_size-2, (bs, tgt_seq_len))
result = generated_lm._generate_beam_search(tgt, **generate_args)
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= tgt_seq_len

### generate

In [ ]:
# export
@patch
@torch.no_grad()
def generate(
    self:GeneratedLM,
    tgt,
    max_length=20,
    do_sample=False,
    num_beams=1,
    temperature=1.0,
    top_k=1,
    top_p=1.0,
    repetition_penalty=1,
    pad_token_id=None,
    eos_token_ids=None,
    length_penalty=1.0,
    model_otherargs=[],
    model_otherkwargs={},
):
    '''
    tgt: (b, tgt_seq_len)
    model_otherargs: Other positional args that your model need. Maybe momory from encoder.
    model_otherkwargs: Other keyword args that your model need. Maybe some masks.
    returns: (b, (tgt_seq_len <= ? <= max_length))
    '''
    assert isinstance(max_length, int) and max_length > 0, "`max_length` should be a strictely positive integer."
    assert isinstance(do_sample, bool), "`do_sample` should be a boolean."
    assert isinstance(num_beams, int) and num_beams > 0, "`num_beams` should be a strictely positive integer."
    assert temperature > 0, "`temperature` should be strictely positive."
    assert isinstance(top_k, int) and top_k >= 0, "`top_k` should be a positive integer."
    assert 0 <= top_p <= 1, "`top_p` should be between 0 and 1."
    assert repetition_penalty >= 1.0, "`repetition_penalty` should be >= 1."
    assert isinstance(pad_token_id, int) and pad_token_id >= 0, "`pad_token_id` should be a positive integer."
    assert isinstance(eos_token_ids, (list, tuple)) and (
        e >= 0 for e in eos_token_ids
    ), "`eos_token_ids` should be a positive integer or a list/tuple of positive integers."
    assert length_penalty > 0, "`length_penalty` should be strictely positive."



    if num_beams > 1:
        output = self._generate_beam_search(
            tgt,
            max_length,
            do_sample,
            temperature,
            top_k,
            top_p,
            repetition_penalty,
            pad_token_id,
            eos_token_ids,
            length_penalty,
            num_beams,
            self.vocab_size,
            model_otherargs,
            model_otherkwargs,
        )
    else:
        output = self._generate_no_beam_search(
            tgt,
            max_length,
            do_sample,
            temperature,
            top_k,
            top_p,
            repetition_penalty,
            pad_token_id,
            eos_token_ids,
            model_otherargs,
            model_otherkwargs,
        )

    return output

In [ ]:
max_length=20
generate_args = dict(   
    max_length=max_length,
    do_sample=True,
    num_beams=1,
    temperature=0.1,
    top_k=3,
    top_p=0.5,
    repetition_penalty=1.2,
    pad_token_id=pad_token_id,
    eos_token_ids=[eos_token_id],
    length_penalty=1,
)

generated_decoder = GeneratedLM(decoder, tgt_vocab_size, support_past=False)
tgt = torch.zeros((bs, 1), dtype=torch.long).fill_(bos_token_id)
result = generated_decoder.generate(tgt, **generate_args, model_otherargs=[memory])
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= 1

generated_lm = GeneratedLM(lm, tgt_vocab_size, support_past=True)
tgt = torch.randint(0, tgt_vocab_size-2, (bs, tgt_seq_len))
result = generated_lm.generate(tgt, **generate_args)
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= tgt_seq_len

generated_lm = GeneratedLM(lm, tgt_vocab_size, support_past=False)
tgt = torch.randint(0, tgt_vocab_size-2, (bs, tgt_seq_len))
result = generated_lm.generate(tgt, **generate_args)
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= tgt_seq_len

In [ ]:
max_length=20
generate_args = dict(   
    max_length=max_length,
    do_sample=True,
    num_beams=2,
    temperature=0.1,
    top_k=3,
    top_p=0.5,
    repetition_penalty=1.2,
    pad_token_id=pad_token_id,
    eos_token_ids=[eos_token_id],
    length_penalty=1,
)

# With memory, Without past
generated_decoder = GeneratedLM(decoder, tgt_vocab_size, support_past=False)
tgt = torch.zeros((bs, 1), dtype=torch.long).fill_(bos_token_id)
result = generated_decoder.generate(tgt, **generate_args, model_otherargs=[memory])
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= 1

# Without memory, With past
generated_lm = GeneratedLM(lm, tgt_vocab_size, support_past=True)
tgt = torch.randint(0, tgt_vocab_size-2, (bs, tgt_seq_len))
result = generated_lm.generate(tgt, **generate_args)
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= tgt_seq_len

# Without memory, Without past
generated_lm = GeneratedLM(lm, tgt_vocab_size, support_past=False)
tgt = torch.randint(0, tgt_vocab_size-2, (bs, tgt_seq_len))
result = generated_lm.generate(tgt, **generate_args)
test_eq(result.shape[0], bs)
assert result.shape[1] <= max_length and result.shape[1] >= tgt_seq_len

## Test

Test that with do_sample=False, GeneratedLM.generate should returns the same result as huggingface's PretrainedModel.generate

In [ ]:
# slow
from transformers import AutoModelWithLMHead, AutoTokenizer
lm = AutoModelWithLMHead.from_pretrained('distilgpt2')
lm.eval()
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

In [ ]:
# slow
sentence = 'The dog is a'
tgt = torch.tensor([tokenizer.encode(sentence)])

generate_args = dict(   
    max_length=20,
    do_sample=False,
    num_beams=1,
    temperature=1.0,
    repetition_penalty=1,
    pad_token_id=0,
    eos_token_ids=[tokenizer.eos_token_id],
    length_penalty=1.0,
)
generated_lm = GeneratedLM(lm, tokenizer.vocab_size, True)
numeric_result = generated_lm.generate(tgt, **generate_args)
result = tokenizer.decode(numeric_result[0])

huggingface_numeric_result = lm.generate(tgt, **generate_args)
huggingface_result = tokenizer.decode(huggingface_numeric_result[0])

test_eq(result, huggingface_result)

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_general.ipynb.
Converted 01a_transforms.ipynb.
Converted 01b_model_splits.ipynb.
Converted 01c_callbacks.ipynb.
Converted 02_generated_lm.ipynb.
Converted 99a_example_roberta_classification.ipynb.
Converted 99b_example_gpt2_lm.ipynb.
Converted index.ipynb.
